In [88]:
import mysql.connector
import pandas as pd
host = "127.0.0.1"          
user = "root"      
password = "Sharath45"  
database = "jorie_ohdsi" 
def fetch_data_from_mysql( query):
    try:
        connection = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )
        
        if connection.is_connected():
            print("Connected to MySQL database")

            # Fetch data into a DataFrame
            df = pd.read_sql(query, connection)

            return df

    except mysql.connector.Error as e:
        print(f"Error connecting to MySQL: {e}")
        return None

    finally:
        # Close the connection
        if 'connection' in locals() and connection.is_connected():
            connection.close()
            print("MySQL connection closed")




In [89]:
query="""select birth_datetime,ethnicity_source_value, race_source_value, person_id,
       gender_source_value,location.address_1,location.zip from person 
       left join location
       on location.location_id=person.person_id
       

"""
person=fetch_data_from_mysql( query)

Connected to MySQL database
MySQL connection closed


C:\Users\Pc\AppData\Local\Temp\ipykernel_9760\4182301056.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [90]:
observation="select * from observation"
observations=fetch_data_from_mysql( observation)
pivoted_df = observations.pivot(index='person_id', columns='observation_type_concept_id', values='value_as_string')
pivoted_df.reset_index(inplace=True)
person=person.merge(pivoted_df,on="person_id",how="left")

Connected to MySQL database
MySQL connection closed


C:\Users\Pc\AppData\Local\Temp\ipykernel_9760\4182301056.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [91]:
person

,birth_datetime,ethnicity_source_value,race_source_value,person_id,gender_source_value,address_1,zip,Alcohol,Smoking
0,*******,Not Hispanic or Latino,White or Caucasian,E223702,Female,3083 N GEORGIA ST,46760,Never,Never; Passive Exposure: Past
1,*******,Not Hispanic or Latino,White or Caucasian,E371944,Male,5858 N 600 E,46731,No,"Former (Quit Date: 1/1/1980), 0 ppd, 0 pack-years"
2,*******,Not Hispanic or Latino,White or Caucasian,E435319,Female,PO BOX 145,46974-0145,No,Never
3,*******,Not Hispanic or Latino,White or Caucasian,E475126,Female,1543 TALLGRASS TRL,46804,Not Currently,Never
4,*******,Not Hispanic or Latino,White or Caucasian,E494826,Male,6321 BENNINGTON DR,46815,Not Currently,"Some Days, 0.25 ppd"
...,...,...,...,...,...,...,...,...,...
2432,*******,Not Hispanic or Latino,White or Caucasian,E994516,Male,1861 GLENLAKE CT,60451,No,"Former (Quit Date: 9/10/2009), 0 ppd, 17.5 pac..."
2433,*******,Not Hispanic or Latino,White or Caucasian,E994965,Male,142 NORTH DRIVE,46740,Not Currently,"Former (Quit Date: 2022), 0 ppd"
2434,*******,Not Hispanic or Latino,White or Caucasian,E99509,Female,525 N BROADWAY ST,46721,No,Never
2435,*******,Not Hispanic or Latino,Patient Declined,E99554,Male,3425 NEW HAVEN AVE,46803,1.0 standard drink of alcohol/week,"Every Day, 0 ppd, 10 pack-years"


In [92]:
smoking_mapping=pd.read_excel(r"..\mappings\Encounter_mapping.xlsx",sheet_name="Smoking")
Alcohol_mapping=pd.read_excel(r"..\mappings\Encounter_mapping.xlsx",sheet_name="Alcohol")


In [93]:
person=person.merge(smoking_mapping,on="Smoking",how="left")

person=person.drop(columns="Smoking")
person=person.rename(columns={"Class":"Smoking"})

In [94]:
person=person.merge(Alcohol_mapping,on="Alcohol",how="left")
person=person.drop(columns="Alcohol")
person=person.rename(columns={"class":"Alcohol"})

In [95]:
person

,birth_datetime,ethnicity_source_value,race_source_value,person_id,gender_source_value,address_1,zip,Smoking,Alcohol
0,*******,Not Hispanic or Latino,White or Caucasian,E223702,Female,3083 N GEORGIA ST,46760,Former Smoker,Never Consumed
1,*******,Not Hispanic or Latino,White or Caucasian,E371944,Male,5858 N 600 E,46731,Former Smoker,Never Consumed
2,*******,Not Hispanic or Latino,White or Caucasian,E435319,Female,PO BOX 145,46974-0145,Never Smoked,Never Consumed
3,*******,Not Hispanic or Latino,White or Caucasian,E475126,Female,1543 TALLGRASS TRL,46804,Never Smoked,Former Drinker
4,*******,Not Hispanic or Latino,White or Caucasian,E494826,Male,6321 BENNINGTON DR,46815,Active Smoker,Former Drinker
...,...,...,...,...,...,...,...,...,...
2432,*******,Not Hispanic or Latino,White or Caucasian,E994516,Male,1861 GLENLAKE CT,60451,Former Smoker,Never Consumed
2433,*******,Not Hispanic or Latino,White or Caucasian,E994965,Male,142 NORTH DRIVE,46740,Former Smoker,Former Drinker
2434,*******,Not Hispanic or Latino,White or Caucasian,E99509,Female,525 N BROADWAY ST,46721,Never Smoked,Never Consumed
2435,*******,Not Hispanic or Latino,Patient Declined,E99554,Male,3425 NEW HAVEN AVE,46803,Active Smoker,Active Drinker


In [96]:
person['zip'] = person['zip'].str.split('-').str[0].astype(int)

In [97]:
Location_mapping=pd.read_excel(r"..\mappings\RUCA2010zipcode.xlsx")


In [98]:
person=person.merge(Location_mapping[["ZIP_CODE","STATE","Category"]],left_on="zip",right_on="ZIP_CODE",how="left")
person=person.drop(columns="ZIP_CODE")
person=person.rename(columns={"Category":"Location_area"})

In [99]:
person.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2437 entries, 0 to 2436
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   birth_datetime          2437 non-null   object
 1   ethnicity_source_value  2402 non-null   object
 2   race_source_value       2404 non-null   object
 3   person_id               2437 non-null   object
 4   gender_source_value     2437 non-null   object
 5   address_1               2437 non-null   object
 6   zip                     2437 non-null   int64 
 7   Smoking                 2367 non-null   object
 8   Alcohol                 2330 non-null   object
 9   STATE                   2437 non-null   object
 10  Location_area           2437 non-null   object
dtypes: int64(1), object(10)
memory usage: 209.6+ KB


In [100]:
encounters="""SELECT 
    person_id, 
    `Encounter Type` AS encounter_type, 
    procedure_date AS encounter_date 
FROM 
    procedure_occurrence

UNION ALL

SELECT 
    person_id, 
    `Encounter Type` AS encounter_type, 
    condition_start_datetime AS encounter_date 
FROM 
    condition_occurrence

UNION ALL

SELECT 
    person_id, 
    visit_type_concept_id AS encounter_type, 
    visit_start_date AS encounter_date 
FROM 
    visit_occurrence;
"""
encounters=fetch_data_from_mysql( encounters)

Connected to MySQL database


C:\Users\Pc\AppData\Local\Temp\ipykernel_9760\4182301056.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


MySQL connection closed


In [101]:
encounters['encounter_date'] = pd.to_datetime(encounters['encounter_date'], format='%m/%d/%Y')

In [102]:
encounters_filtered=encounters[encounters["encounter_date"]<="2024-11-27"]

In [105]:
# Getting the latest and earliest encounter date for each person
index_date = encounters_filtered.groupby(['person_id'])['encounter_date'].max().reset_index(name='latest_encounter_date')
earliest_date = encounters_filtered.groupby(['person_id'])['encounter_date'].min().reset_index(name='earliest_encounter_date')
patient_dates = index_date.merge(earliest_date, on="person_id", how="left")


In [106]:
patient_dates["latest_encounter_date"].min()
patient_dates["earliest_encounter_date"].min()


Timestamp('2004-04-16 00:00:00')

Conditions

In [107]:
Diabetes_type=pd.read_excel(r"..\mappings\ICD10_diabetes_codes.xlsx")

In [108]:
conditions="""select * from condition_occurrence
"""
conditions=fetch_data_from_mysql( conditions)

Connected to MySQL database


C:\Users\Pc\AppData\Local\Temp\ipykernel_9760\4182301056.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


MySQL connection closed


In [109]:
conditions_patinet_level=conditions[["person_id","condition_type_concept_id"]].drop_duplicates()

In [110]:
conditions_patinet_level

,person_id,condition_type_concept_id
0,E379593,None
2,E509556,None
14,E509556,M48.061
32,E1130335,None
62,E522307,None
...,...,...
398093,E1681969,E11.9
398239,E692445,T84.84XA
398240,E692445,E11.9
398299,E2084796,E11.9


In [111]:
conditions_patinet_level["condition_type_concept_id"].value_counts()

condition_type_concept_id
E11.9       1042
Z79.4        486
E66.01       365
I73.9        311
E11.42       230
            ... 
E08.9          1
T87.43         1
S12.200A       1
C13.9          1
E11.3219       1
Name: count, Length: 1531, dtype: int64

In [112]:
diabtetic_patients=conditions[conditions["condition_type_concept_id"].isin(Diabetes_type["ICD_10"].unique())]



In [113]:
diabtetic_patients["condition_start_datetime"]=pd.to_datetime(diabtetic_patients["condition_start_datetime"])
Diabetic_encounter_date=diabtetic_patients.groupby(['person_id'])['condition_start_datetime'].min().reset_index(name='Diabetes_encounter_date')

C:\Users\Pc\AppData\Local\Temp\ipykernel_9760\201268862.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diabtetic_patients["condition_start_datetime"]=pd.to_datetime(diabtetic_patients["condition_start_datetime"])


In [114]:

patient_dates=patient_dates.merge(Diabetic_encounter_date,on="person_id",how="left")


In [115]:
patient_dates

,person_id,latest_encounter_date,earliest_encounter_date,Diabetes_encounter_date
0,E1000022,2024-11-12,2024-08-04,NaT
1,E100202,2020-04-29,2017-09-10,2017-09-10
2,E1003429,2024-11-12,2024-05-22,2024-05-22
3,E1004123,2019-11-27,2018-10-03,2018-10-03
4,E1004509,2023-08-17,2011-02-04,2011-02-04
...,...,...,...,...
2396,E994516,2023-03-14,2021-06-22,2021-06-22
2397,E994965,2024-11-18,2024-04-23,2024-04-23
2398,E99509,2024-10-21,2020-09-11,2020-09-11
2399,E99554,2024-11-01,2024-07-25,2024-07-25


In [116]:
diabtetic_patients=diabtetic_patients.merge(Diabetes_type,left_on="condition_type_concept_id",right_on="ICD_10",how="left")

In [118]:
diabtetic_patients["Type"].unique()

array(['Type-2', 'Other specified diabetes mellitus',
       'DM due to underlying condition', 'Type-1',
       'Drug or Chemical-Induced Diabetes Mellitus'], dtype=object)

In [119]:
person

,birth_datetime,ethnicity_source_value,race_source_value,person_id,gender_source_value,address_1,zip,Smoking,Alcohol,STATE,Location_area
0,*******,Not Hispanic or Latino,White or Caucasian,E223702,Female,3083 N GEORGIA ST,46760,Former Smoker,Never Consumed,IN,small town
1,*******,Not Hispanic or Latino,White or Caucasian,E371944,Male,5858 N 600 E,46731,Former Smoker,Never Consumed,IN,small town
2,*******,Not Hispanic or Latino,White or Caucasian,E435319,Female,PO BOX 145,46974,Never Smoked,Never Consumed,IN,Micropolitan
3,*******,Not Hispanic or Latino,White or Caucasian,E475126,Female,1543 TALLGRASS TRL,46804,Never Smoked,Former Drinker,IN,Metropolitan
4,*******,Not Hispanic or Latino,White or Caucasian,E494826,Male,6321 BENNINGTON DR,46815,Active Smoker,Former Drinker,IN,Metropolitan
...,...,...,...,...,...,...,...,...,...,...,...
2432,*******,Not Hispanic or Latino,White or Caucasian,E994516,Male,1861 GLENLAKE CT,60451,Former Smoker,Never Consumed,IL,Metropolitan
2433,*******,Not Hispanic or Latino,White or Caucasian,E994965,Male,142 NORTH DRIVE,46740,Former Smoker,Former Drinker,IN,small town
2434,*******,Not Hispanic or Latino,White or Caucasian,E99509,Female,525 N BROADWAY ST,46721,Never Smoked,Never Consumed,IN,small town
2435,*******,Not Hispanic or Latino,Patient Declined,E99554,Male,3425 NEW HAVEN AVE,46803,Active Smoker,Active Drinker,IN,Metropolitan


In [120]:
import numpy as np

# Loop through each unique "Type" in the Diabetes_type dataframe
for diabetes_type in Diabetes_type["Type"].unique():
    # Filter the Diabetes_type dataframe for the current diabetes type
    current_type_df = Diabetes_type[Diabetes_type["Type"] == diabetes_type]
    
    # Filter the diabetic_patients dataframe for the ICD_10 codes matching the current type
    current_type_patients = diabtetic_patients[diabtetic_patients["ICD_10"].isin(current_type_df["ICD_10"])]
    
    # Create a new column for the specific diabetes type in the person dataframe
    person[diabetes_type] = np.where(person["person_id"].isin(current_type_patients["person_id"]), 1, 0)


In [122]:
Comorbidities=pd.read_excel("..\mappings\ICD_CCI_CODES.xlsx")
Comorbidities.columns

<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\m'
C:\Users\Pc\AppData\Local\Temp\ipykernel_9760\2090531556.py:1: SyntaxWarning: invalid escape sequence '\m'
  Comorbidities=pd.read_excel("..\mappings\ICD_CCI_CODES.xlsx")


Index(['ICD_Code', 'ICD_Description', 'Category'], dtype='object')

In [124]:

# Loop through unique values in the 'Column Name/Label' column of the Comorbidities table
for i in Comorbidities['Category'].unique():
    # Get codes for i from 'Concept Code' column in Comorbidities table
    codes = Comorbidities[Comorbidities['Category'] == i]['ICD_Code'].unique()
    
    # Filter the Conditions table based on the retrieved codes
    filtered_conditions = conditions[conditions['condition_type_concept_id'].isin(codes)]
    
    # Get the unique list of patient IDs from the filtered Conditions table
    unique_patients = filtered_conditions['person_id'].unique()
    
    # Create a new column in Patients_transformed with the same name as i
    # Set the value to 1 if the patient ID is in the unique_patients list, otherwise set it to 0
    person[i] = person['person_id'].apply(lambda x: 1 if x in unique_patients else 0)
    print(i,len(unique_patients),len(filtered_conditions))

LIVER_MILD 11 322
LIVER 8 242
AIDS 3 116
MALIG 129 4196
METAS 8 300
DIAB_CHRO 92 2649
DEMENTIA 20 562
CEREBRO 104 3520
HEMIP 0 0
CHF 69 2181
RENAL 138 5317
MYOC 74 2217
PVD 329 10087
COPD 272 8264
PEPTIC 2 2
RHEUM 47 1455


In [129]:
visit_occurence="""select * from visit_occurrence
"""
visit_occurence=fetch_data_from_mysql( visit_occurence)

Connected to MySQL database
MySQL connection closed


C:\Users\Pc\AppData\Local\Temp\ipykernel_9760\4182301056.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [130]:
visit_occurence["visit_start_datetime"]=pd.to_datetime(visit_occurence["visit_start_datetime"])

In [131]:
visits_filtered=visit_occurence[visit_occurence["visit_start_datetime"]<="2024-11-27"]

In [132]:
visits_filtered=visits_filtered.merge(patient_dates[['person_id', 'latest_encounter_date']],on="person_id",how="left")

In [133]:
visits_filtered["Days"]=(visits_filtered["latest_encounter_date"]-visits_filtered["visit_start_datetime"]).dt.days

In [135]:
visits_filtered['visit_type_concept_id'] = visits_filtered['visit_type_concept_id'].replace({
    'Admission (Discharged)': 'IP',
    'Office Visit': 'OP'
})


In [136]:
visits_filtered['visit_type_concept_id'].unique()

array(['IP', 'ED', 'OP', 'Surgery'], dtype=object)

In [138]:
# List of encounter types to process
encounter_types = visits_filtered["visit_type_concept_id"].unique()

# Initialize the merged dataframe (assuming 'person' is your base DataFrame)
for encounter_type in encounter_types:
    # Filter data for the current encounter type
    encounter_data = visits_filtered[visits_filtered["visit_type_concept_id"] == encounter_type]
    
    # Iterate over the different time periods (30 and 365 days)
    for days in [30, 365]:
        # Filter for the specific time window
        encounter_data_filtered = encounter_data[(encounter_data["Days"] >= 0) & (encounter_data["Days"] <= days)]
        
        # Group by 'person_id' and count unique visit start datetimes
        encounter_data_grouped = encounter_data_filtered.groupby('person_id')['visit_start_datetime'].nunique().reset_index()
        
        # Create a proper column name without extra characters
        column_name = f"{encounter_type.replace(' ', '_')}_{days}"
        
        # Rename the column to reflect the encounter type and time period
        encounter_data_grouped = encounter_data_grouped.rename(columns={"visit_start_datetime": column_name})
        
        # Merge the results into the 'person' dataframe
        person = person.merge(encounter_data_grouped[['person_id', column_name]], 
                              on='person_id', how='left')


In [139]:
person.columns

Index(['birth_datetime', 'ethnicity_source_value', 'race_source_value',
       'person_id', 'gender_source_value', 'address_1', 'zip', 'Smoking',
       'Alcohol', 'STATE', 'Location_area', 'DM due to underlying condition',
       'Drug or Chemical-Induced Diabetes Mellitus', 'Type-1', 'Type-2',
       'Other specified diabetes mellitus',
       'Diabetes Mellitus in Pregnancy, Childbirth, and the Puerperium',
       'LIVER_MILD', 'LIVER', 'AIDS', 'MALIG', 'METAS', 'DIAB_CHRO',
       'DEMENTIA', 'CEREBRO', 'HEMIP', 'CHF', 'RENAL', 'MYOC', 'PVD', 'COPD',
       'PEPTIC', 'RHEUM', 'IP_30', 'IP_365', 'ED_30', 'ED_365', 'OP_30',
       'OP_365', 'Surgery_30', 'Surgery_365'],
      dtype='object')

In [140]:
procedure_occurence="""select * from procedure_occurrence
"""
procedure_occurence=fetch_data_from_mysql( procedure_occurence)

Connected to MySQL database
MySQL connection closed


C:\Users\Pc\AppData\Local\Temp\ipykernel_9760\4182301056.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [141]:
procedure_occurence.columns
procedure_occurence["procedure_date"]=pd.to_datetime(procedure_occurence["procedure_date"])

In [142]:
filtered_procedures=procedure_occurence[procedure_occurence["procedure_date"]<="2024-11-27"]

In [143]:
CPT_codes=pd.read_excel("../mappings/All_CPT_codes.xlsx",sheet_name="ALL 2024 CPT Codes")

In [78]:
filtered_procedures.columns

Index(['person_id', 'Encounter Type', 'procedure_date', 'Mapped Proc Code'], dtype='object')

In [144]:
filtered_procedures["Mapped Proc Code"].value_counts()[:15]

Mapped Proc Code
         104
45378     64
64483     46
43235     40
43239     39
27447     31
64721     20
45380     19
45385     19
27096     18
27130     17
26055     13
22853     12
20680     10
23412     10
Name: count, dtype: int64

In [146]:
import pandas as pd
import numpy as np

# Function for procedures filtering
def procedures_filtering(table_column, column):
    df = filtered_procedures[filtered_procedures["Mapped Proc Code"].isin(table_column.unique())]
    person[column] = np.where(person["person_id"].isin(df["person_id"]), 1, 0)

# Function for conditions filtering
def conditions_filtering(table_column, column):
    df = conditions_patinet_level[conditions_patinet_level["condition_type_concept_id"].isin(table_column.unique())]
    person[column] = np.where(person["person_id"].isin(df["person_id"]), 1, 0)

# Read the Excel file with all the sheets
excel_file = "..\\mappings\\Condition_codes.xlsx"
xls = pd.ExcelFile(excel_file)

# Loop through each sheet in the Excel file
for sheet_name in xls.sheet_names:
    sheet = pd.read_excel(excel_file, sheet_name=sheet_name)
    
    # Check if the sheet has 'CPT' column
    if 'CPT' in sheet.columns:
        procedures_filtering(sheet['CPT'], sheet_name)
    
    # Check if the sheet has 'ICD' column
    elif 'ICD' in sheet.columns:
        conditions_filtering(sheet['ICD'], sheet_name)
    else:
        print(f"No CPT or ICD column found in sheet: {sheet_name}")


No CPT or ICD column found in sheet: Insurance type
No CPT or ICD column found in sheet: ED


In [148]:
len(person.columns)

80

In [150]:
CPT_codes=pd.read_excel("../mappings/All_CPT_codes.xlsx",sheet_name="ALL 2024 CPT Codes")

In [151]:
# Loop through unique values in the 'Category' column of the CPT_codes table
for i in CPT_codes['Category'].unique():
    # Get codes for i from 'CPT Codes' column in CPT_codes table
    codes = CPT_codes[CPT_codes['Category'] == i]['CPT Codes'].astype(str).unique()  # Convert to string
    
    # Filter the filtered_procedures table based on the retrieved codes
    procs = filtered_procedures[filtered_procedures['Mapped Proc Code'].astype(str).isin(codes)]  # Convert to string
    
    # Get the unique list of patient IDs from the filtered procedures table
    unique_patients = procs['person_id'].unique()
    
    # Ensure 'person_id' in the Patients_transformed table is the same data type (convert to string)
    person['person_id'] = person['person_id'].astype(str)
    
    # Create a new column in Patients_transformed with the same name as i
    # Set the value to 1 if the patient ID is in the unique_patients list, otherwise set it to 0
    person[i] = person['person_id'].apply(lambda x: 1 if x in unique_patients else 0)
    
    print(i, len(unique_patients), len(procs))


Abdominal aortic aneurysm repair 0 0
Limb amputation 30 32
Appendix surgery 3 4
AV shunt for dialysis  5 6
Bile duct, liver or pancreatic surgery 0 0
Breast surgery 2 2
Cardiac surgery 0 0
Coronary bypass with chest & donor incisions 0 0
Coronary bypass graft with chest incision 0 0
Carotid endarterectomy 0 0
Gallbladder surgery 2 4
Colon surgery 0 0
Craniotomy 0 0
Cesarean section 0 0
Spinal fusion 17 22
Open reduction of fracture 13 18
Gastric surgery 0 0
Herniorrhaphy 1 1
Hip prosthesis 21 22
Heart transplant 0 0
Abdominal hysterectomy 1 1
Knee prosthesis 30 34
Kidney transplant 0 0
Laminectomy 6 6
Liver transplant 0 0
Neck surgery 0 0
Kidney surgery 5 7
Ovarian surgery 2 2
Pacemaker surgery 1 2
Prostate surgery 0 0
Peripheral vascular bypass surgery 1 1
Rectal surgery  0 0
Small bowel surgery 0 0
Spleen surgery 0 0
Thoracic surgery  0 0
Thyroid and/or parathyroid surgery 0 0
Vaginal hysterectomy 0 0
Ventricular shunt 0 0
Exploratory laparotomy 5 9


In [ ]:
person.to_excel("Data\Output\Patients_features.xlsx")

In [153]:
constant_columns = [col for col in person.columns if person[col].nunique() == 1]
num_constant_columns = len(constant_columns)
num_constant_columns


45